In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import missingno as msno

In [ ]:
%matplotlib inline  
style.use('fivethirtyeight') 
sns.set(style='whitegrid',color_codes=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder

from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification

In [ ]:
df=pd.read_csv(r'Data/winequality.csv')

In [ ]:
df.shape

In [ ]:
df.head(20)

In [ ]:
df.columns 

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
msno.matrix(df)

In [ ]:
df.describe(include='all')

In [ ]:
sns.factorplot(data=df,kind='box',size=10,aspect=2.5)

In [ ]:
fig,axes=plt.subplots(5,5)
columns=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality']
for i in range (5):
    for j in range (5):
        axes[i,j].hist(x=columns[i+j],data=df,edgecolor='#000000',linewidth=2,color='#ff4125')
        axes[i,j].set_title('Variation of '+columns[i+j])
fig=plt.gcf()
fig.set_size_inches(18,18)
fig.tight_layout()

In [ ]:
cor_mat= df.corr()
mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False
fig=plt.gcf()
fig.set_size_inches(30,12)
sns.heatmap(data=cor_mat,mask=mask,square=True,annot=True,cbar=True)

In [ ]:
def plot(feature_x,target='quality'):
    sns.factorplot(x=target,y=feature_x,data=df,kind='bar',size=5,aspect=1)
    sns.factorplot(x=target,y=feature_x,data=df,kind='violin',size=5,aspect=1)
    sns.factorplot(x=target,y=feature_x,data=df,kind='swarm',size=5,aspect=1)

In [ ]:
plot('fixed acidity','quality')

In [ ]:
plot('alcohol','quality')

In [ ]:
bins = (2, 6.5, 8)
group_names = ['bad', 'good']
df['quality'] = pd.cut(df['quality'], bins = bins, labels = group_names)

In [ ]:
label_quality = LabelEncoder()

In [ ]:
df['quality'] = label_quality.fit_transform(df['quality'])

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('quality',axis=1),df['quality'],test_size=0.25,random_state=42)

In [ ]:
models=[LogisticRegression(),LinearSVC(),SVC(kernel='rbf'),KNeighborsClassifier(),RandomForestClassifier(),DecisionTreeClassifier(),GradientBoostingClassifier(),GaussianNB()]

model_names=['LogisticRegression','LinearSVM','rbfSVM','KNearestNeighbors','RandomForestClassifier','DecisionTree','GradientBoostingClassifier','GaussianNB']

In [ ]:
acc=[]
d={}

for model in range(len(models)):
    clf=models[model]
    clf.fit(x_train,y_train)
    pred=clf.predict(x_test)
    acc.append(accuracy_score(pred,y_test))

d={'Modeling Algo':model_names,'Accuracy':acc}
d

In [ ]:
acc_frame=pd.DataFrame(d)
acc_frame

In [ ]:
sns.barplot(y='Modeling Algo',x='Accuracy',data=acc_frame)

In [ ]:
sns.factorplot(x='Modeling Algo',y='Accuracy',data=acc_frame,kind='point',size=4,aspect=3.5)

In [ ]:
acc_frame

In [ ]:
def scale_func(x_train,x_test,y_train,y_test,name_scaler):
    models=[LogisticRegression(),LinearSVC(),SVC(kernel='rbf'),KNeighborsClassifier(),RandomForestClassifier(),DecisionTreeClassifier(),GradientBoostingClassifier(),GaussianNB()]
    acc_sc=[]
    x_train_scale = name_scaler.fit_transform(x_train)
    x_test_scale = name_scaler.transform(x_test)

    for model in range(len(models)):
        clf=models[model]
        clf.fit(x_train_scale,y_train)
        pred=clf.predict(x_test_scale)
        acc_sc.append(accuracy_score(pred,y_test))
     
    acc_frame["Standard Scaler"]=np.array(acc_sc)


In [ ]:
scale_func(x_train,x_test,y_train,y_test,StandardScaler())

In [ ]:
acc_frame

In [ ]:
sns.barplot(y='Modeling Algo',x='Standard Scaler',data=acc_frame)

# Parameter Tuning
## Logistic Regression

In [ ]:
params_dict={'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],'penalty':['l1','l2']}
clf_lr=GridSearchCV(estimator=LogisticRegression(),param_grid=params_dict,scoring='accuracy',cv=10)
clf_lr.fit(x_train,y_train)

In [ ]:
clf_lr.best_params_

In [ ]:
clf_lr.best_score_

In [ ]:
pred=clf_lr.predict(x_test)
accuracy_score(pred,y_test)

## k-Nearest Neighbor (KNN)

In [ ]:
l=[i+1 for i in range(50)]
params_dict={'n_neighbors':l,'n_jobs':[-1]}
clf_knn=GridSearchCV(estimator=KNeighborsClassifier(),param_grid=params_dict,scoring='accuracy',cv=10)
clf_knn.fit(x_train,y_train)

In [ ]:
clf_knn.best_params_

In [ ]:
clf_knn.best_score_

In [ ]:
pred=clf_knn.predict(x_test)
accuracy_score(pred,y_test)

## Support Vector Machine - SVM

In [ ]:
params_dict={'C':[0.001,0.01,0.1,1,10,100],'gamma':[0.001,0.01,0.1,1,10,100],'kernel':['linear','rbf']}
clf=GridSearchCV(estimator=SVC(),param_grid=params_dict,scoring='accuracy',cv=10)
clf.fit(x_train,y_train)

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
# now tuning finally around these values of C and gamma and the kernel for further increasing the accuracy.
params_dict={'C':[0.90,0.92,0.96,0.98,1.0,1.2,1.5],'gamma':[0.90,0.92,0.96,0.98,1.0,1.2,1.5],'kernel':['linear','rbf']}
clf_svm=GridSearchCV(estimator=SVC(),param_grid=params_dict,scoring='accuracy',cv=10)
clf_svm.fit(x_train,y_train)

In [ ]:
clf_svm.best_score_

In [ ]:
clf_svm.best_params_

In [ ]:
clf_svm.grid_scores_

In [ ]:
pred=clf_svm.predict(x_test)
accuracy_score(pred,y_test)   # actual accuarcy on our test set.

## Random Forest

In [ ]:
params_dict={'n_estimators':[500],'max_features':['auto','sqrt','log2']}
clf_rf=GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1),param_grid=params_dict,scoring='accuracy',cv=10)
clf_rf.fit(x_train,y_train)

In [ ]:
clf_rf.best_score_

In [ ]:
clf_rf.best_params_

In [ ]:
clf_rf.grid_scores_

In [ ]:
pred=clf_rf.predict(x_test)
accuracy_score(pred,y_test)   # actual accuarcy on our test set.

## Gradient Boosting

In [ ]:
clf_gb=GridSearchCV(estimator=GradientBoostingClassifier(),cv=10,param_grid=dict({'n_estimators':[500]}))
clf_gb.fit(x_train,y_train)

In [ ]:
clf_gb.best_score_

In [ ]:
clf_gb.best_params_

In [ ]:
pred=clf_gb.predict(x_test)
accuracy_score(pred,y_test)